# Intro

This notebook serves as a tiny example on how to train a DNN using TensorFlow and export to the NNet format that NeuralVerification.jl can parse.

# Requirements

Before we begin make sure your environment satisfies the requirements specified in `requirements.txt`, the easiest way is to create a new virtualenv and run:

`pip install -r requirements.txt`

then you can go ahead and run the following cells, for this make sure that your jupyter notebook is using the kernel corresponding to the virtualenv you just created (Chris: should I expand on this?)

# imports needed

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import math
import numpy as np

# fetch and load the data

In [ ]:
mnist = input_data.read_data_sets('./mnist_data')
model_name = "mnist_network"

# define auxiliary functions

In [ ]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# define the TF variables

In [ ]:
INPUT = 784 # mnist images are 28 x 28 = 784 pixels
OUTPUT = 10 # there are 10 possible classes for each image (0-9 digits)
HFC = 50 # hidden layer size

# placeholder used to feed in the input, it is very important to explicitly name
# which will enable the script that generates the nnet file from the protobuffer
# to identify the relevant part of the computational graph

x_in = tf.placeholder(tf.float32, [None, INPUT], name='input_op')

W_fc = weight_variable([INPUT, HFC])
b_fc = bias_variable([HFC])

W_fc1 = weight_variable([HFC, HFC])
b_fc1 = bias_variable([HFC])

#W_fc2 = weight_variable([HFC, HFC])
#b_fc2 = bias_variable([HFC])

#W_fc3 = weight_variable([HFC, HFC])
#b_fc3 = bias_variable([HFC])

#W_fc4 = weight_variable([HFC, HFC])
#b_fc4 = bias_variable([HFC])

#W_fc5 = weight_variable([HFC, HFC])
#b_fc5 = bias_variable([HFC])


W_o = weight_variable([HFC, OUTPUT])
b_o = bias_variable([OUTPUT])

#
x_image = tf.reshape(x_in, [-1, int(math.sqrt(INPUT)), int(math.sqrt(INPUT)), 1])

# define the network

In [ ]:
h_fc = tf.nn.relu(tf.add(tf.matmul(x_in, W_fc), b_fc))
h_fc1 = tf.nn.relu(tf.add(tf.matmul(h_fc, W_fc1), b_fc1))
#h_fc2 = tf.nn.relu(tf.add(tf.matmul(h_fc1, W_fc2), b_fc2))
#h_fc3 = tf.nn.relu(tf.add(tf.matmul(h_fc2, W_fc3), b_fc3))
#h_fc4 = tf.nn.relu(tf.add(tf.matmul(h_fc3, W_fc4), b_fc4))
#h_fc5 = tf.nn.relu(tf.add(tf.matmul(h_fc4, W_fc5), b_fc5))
y = tf.add(tf.matmul(h_fc1, W_o), b_o, name="output_op")


# defining the loss and optimizer

In [ ]:
y_ = tf.placeholder(tf.int64, [None])
cross_entropy = tf.losses.sparse_softmax_cross_entropy(labels=y_, logits=y)
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [ ]:
epochs = 10 # epochs for training
batch_size = 64 # batch size

N = mnist.train.num_examples
batch_amount = epochs*N//batch_size

# Train and save protobuffer

In [ ]:
epochs = 10 # epochs for training
batch_size = 50 # batch size

N = mnist.train.num_examples
batch_amount = epochs*N//batch_size

with tf.Session() as sess:
    epoch_in = 1
    tf.global_variables_initializer().run()
    for i in range(batch_amount):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        sess.run(train_step, feed_dict={x_in: batch_xs, y_: batch_ys})
        if (i*batch_size) % N == 0:
            #print('loss> {}'.format(sess.run(cross_entropy, feed_dict={x_in: batch_xs, y_: batch_ys})))
            print("Epoch {} done.".format(epoch_in))
            epoch_in += 1
        
    correct_prediction = tf.equal(tf.argmax(y, 1), y_)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    print("Accuracy: {}".format(sess.run(accuracy, feed_dict={x_in: mnist.test.images, y_: mnist.test.labels})))
    output_graph_def = tf.graph_util.convert_variables_to_constants(sess, tf.get_default_graph().as_graph_def(), ['output_op']) 
    
    #saver = tf.train.Saver()
    #save_path = saver.save(sess, 'models/'+model_name+'.ckpt')
    with tf.gfile.GFile('models/' + model_name+'.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

# Now let's convert the protobuffer to NNet format

In [ ]:
!python ./NNet/scripts/pb2nnet.py